## **1. Paramètres**

In [10]:
# Importation des bibliothèques
import os
import geopandas as gpd
import fiona
import rasterio
from rasterio import features
from rasterio.transform import from_origin
import numpy as np
from scipy import ndimage

In [11]:
# Définition des chemins de sortie
partial_output_path = '../data_brut/2_rasters'
final_output_path = '../data_final'

# Vérifier et créer les répertoires
if not os.path.exists(partial_output_path):
    os.makedirs(partial_output_path)

if not os.path.exists(final_output_path):
    os.makedirs(final_output_path)  


In [12]:
# Chemins des données
path_bd = "../data_brut/bd_tls.gpkg"
layers = fiona.listlayers(path_bd)

# Afficher la liste des couches
print("Couches disponibles dans le fichier GeoPackage :")
for layer in layers:
    print(layer)

Couches disponibles dans le fichier GeoPackage :
lieu_dit_non_habite
adresse_ban
aerodrome
batiment
canalisation
cimetiere
commune
construction_lineaire
construction_surfacique
cours_d_eau
departement
equipement_de_transport
foret_publique
ligne_electrique
parc_ou_reserve
piste_d_aerodrome
plan_d_eau
region
reservoir
route_numerotee_ou_nommee
surface_hydrographique
terrain_de_sport
toponymie_lieux_nommes
toponymie_zones_reglementees
transport_par_cable
troncon_de_route
troncon_de_voie_ferree
troncon_hydrographique
voie_nommee
zone_d_activite_ou_d_interet
zone_d_habitation
zone_de_vegetation
toulouse_metropole
insee_carreaux_200m
route_cyclabe_geovelo
iris_pop_diplomes
bio_markets
iris_csp
centroides_zones
layer_styles


In [19]:
# Charger les couches de lignes et de département
pistes_cyclables = gpd.read_file(path_bd, layer='route_cyclabe_geovelo')
routes_nommee = gpd.read_file(path_bd, layer='route_numerotee_ou_nommee')
tls_met = gpd.read_file(path_bd, layer='toulouse_metropole')

In [14]:
# Obtenir l'emprise du département
minx, miny, maxx, maxy = tls_met.total_bounds

### **2. Fonction pour calculer les distances**

In [15]:
def calculate_distance(layer, mask_layer, output_name, resolution):
    # Calculer la taille du raster en fonction de la résolution
    n_cols = int(np.ceil((maxx - minx) / resolution))
    n_rows = int(np.ceil((maxy - miny) / resolution))
    transform = from_origin(minx, maxy, resolution, resolution)
    raster_shape = (n_rows, n_cols)
    
    # Rasteriser la couche de lignes
    shapes = ((geom, 1) for geom in layer.geometry)
    rasterized = features.rasterize(
        shapes=shapes,
        out_shape=raster_shape,
        transform=transform,
        fill=0,
        all_touched=True,
        dtype=np.uint8
    )
    
    # Calcul de la distance
    distance = ndimage.distance_transform_edt(1 - rasterized) * resolution
    distance = distance.astype('float32')
    
    # Appliquer le masque du département
    shapes_mask = ((geom, 1) for geom in mask_layer.geometry)
    raster_mask = features.rasterize(
        shapes=shapes_mask,
        out_shape=raster_shape,
        transform=transform,
        fill=0,
        all_touched=True,
        dtype=np.uint8
    )
    distance_masked = np.where(raster_mask == 1, distance, np.nan)
    
    # Enregistrer le raster avec compression LZW
    output_filepath = os.path.join(partial_output_path, output_name)
    compress_options = {
        'compress': 'LZW',
        'predictor': 3,
        'tiled': True,
        'blockxsize': 256,
        'blockysize': 256
    }
    
    with rasterio.open(
        output_filepath,
        'w',
        driver='GTiff',
        height=n_rows,
        width=n_cols,
        count=1,
        dtype='float32',
        crs=layer.crs,
        transform=transform,
        nodata=np.nan,
        **compress_options
    ) as dst:
        dst.write(distance_masked, 1)
    print(f"Le raster est enregistré à l'emplacement : {output_filepath}")

### **3. Calculer les distances et sauvegarder les rasters**

In [16]:
# Calculer et sauvegarder les rasters avec la résolution spécifiée
calculate_distance(pistes_cyclables, tls_met, 'dist_pistes_cyclables_abs.tif', resolution=5)
calculate_distance(routes_nommee, tls_met, 'dist_routes_nommee_abs.tif', resolution=5)

Le raster est enregistré à l'emplacement : ../data_brut/2_rasters\dist_pistes_cyclables_abs.tif
Le raster est enregistré à l'emplacement : ../data_brut/2_rasters\dist_routes_nommee_abs.tif


### **3. Normalisátion des rasters**

In [17]:
# Fonction pour normaliser les rasters en définissant 0 comme NoData
def normaliser_raster(input_file, output_file):
    with rasterio.open(input_file) as src:
        data = src.read(1)
        
        # Remplacer NoData par NaN pour éviter qu'ils n'influencent la normalisation
        data[data == src.nodata] = np.nan
        
        # Calculer les valeurs minimales et maximales en ignorant les NaN
        min_val, max_val = np.nanmin(data), np.nanmax(data)
        
        # Normaliser les données entre 0 et 1
        normalized_data = (data - min_val) / (max_val - min_val)
        
        # Attribuer NaN à tous les pixels avec la valeur 0 après la normalisation
        normalized_data[normalized_data == 0] = np.nan
        
        # Mettre à jour le profil du raster pour définir NaN comme NoData
        profile = src.profile
        profile.update(dtype='float32', nodata=np.nan)
    
    # Sauvegarder le raster normalisé avec 0 comme NoData
    with rasterio.open(output_file, 'w', **profile) as dst:
        dst.write(normalized_data, 1)


In [18]:
# Normaliser les raster
normaliser_raster(f'{partial_output_path}/dist_pistes_cyclables_abs.tif', f'{final_output_path}/dist_pistes_cyclables_norm.tif')
normaliser_raster(f'{partial_output_path}/dist_routes_nommee_abs.tif', f'{final_output_path}/dist_routes_nommee_norm.tif')
